In [20]:
import numpy as np
import keras
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
import data_set_ooc
%load_ext autoreload
%autoreload 2



In [21]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['batch_size'] = 20
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 5
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01
FLAGS['epochs'] = 30

In [38]:
data = data_set_ooc.DataSet(width = FLAGS['width'], height=FLAGS['height'])

In [39]:
bx, by = data.next_batch(FLAGS['batch_size'])

Loading next batch
Read train data time: 1.93 seconds


In [42]:
by.shape

(16, 8)

In [12]:
X_val, y_val = data.get_validation_set()

In [13]:
from keras.callbacks import TensorBoard
tb = TensorBoard(log_dir='keras_tb/', histogram_freq=5, write_graph=True, write_images=False)

In [14]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(32, 32, 3)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


In [15]:
model = create_model()

In [16]:
callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, verbose=0),
tb]
hist = model.fit(data.X_train, data.y_train, batch_size=FLAGS['batch_size'], nb_epoch=FLAGS['epochs'],
          shuffle=True, verbose=2, validation_data=(X_val, y_val),
              callbacks=callbacks)



Train on 3021 samples, validate on 756 samples
INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.
INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.
INFO:tensorflow:Summary name convolution2d_2_W:0 is illegal; using convolution2d_2_W_0 instead.
INFO:tensorflow:Summary name convolution2d_2_b:0 is illegal; using convolution2d_2_b_0 instead.
INFO:tensorflow:Summary name convolution2d_3_W:0 is illegal; using convolution2d_3_W_0 instead.
INFO:tensorflow:Summary name convolution2d_3_b:0 is illegal; using convolution2d_3_b_0 instead.
INFO:tensorflow:Summary name convolution2d_4_W:0 is illegal; using convolution2d_4_W_0 instead.
INFO:tensorflow:Summary name convolution2d_4_b:0 is illegal; using convolution2d_4_b_0 instead.
INFO:tensorflow:Summary name dense_1_W:0 is illegal; using dense_1_W_0 instead.
INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.
INFO:tensorflow:Summary n

In [17]:
X_test, y_test = load_images.load_test(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [18]:
preds = model.predict(X_test, batch_size=FLAGS['batch_size'], verbose=2)


In [24]:
import make_submission
submit = make_submission.makeSubmission(preds,y_test)

In [27]:
submit.to_csv('keras_32_32.csv', index=False)